This notebook will try to explore the effectiveness of prknn vs standard knn models for prediction on an inbalanced dataset.   

In [290]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, TargetEncoder

from sklearn.impute import KNNImputer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

from test_class import PRKNeighborsClassifier
from tqdm import tqdm

In [291]:
RANDOM_STATE = 33

In [292]:
df= pd.read_csv('train_data.csv')

In [293]:
df.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [294]:
columns_to_drop = [
    "Hospital_region_code",
    "Ward_Facility_Code",
    "case_id",
    "patientid",
    "Hospital_code",
    "City_Code_Patient",
    "Admission_Deposit",
    "Available Extra Rooms in Hospital",
    "Bed Grade",
    "City_Code_Hospital",
    "Visitors with Patient",
    "Hospital_type_code",
    "Type of Admission",
    "Department"
]

df = df.drop(columns_to_drop, axis=1)

In [295]:
df.isna().sum()

Ward_Type              0
Severity of Illness    0
Age                    0
Stay                   0
dtype: int64

For categoric variables, I am going to use frequency encoding, due to the high number of categoric features and their high cardinality. I am also concerned that target encoding may lead to data leakage due to the low frequencies of the target variable. While other techniques may be better, the aim is to test the effectiveness of prknn compared to knn.

In [296]:
categoric_columns = [
    'Hospital_type_code',
    'City_Code_Hospital',
    'Department', 
    'Ward_Type',
    'City_Code_Patient', 
    'Type of Admission'
]

ordinal_columns = [
    'Bed Grade',
    'Severity of Illness'
]

cont_columns = [
    'Available Extra Rooms in Hospital',
    'Visitors with Patient',
    'Age',
    'Admission_Deposit'
]

In [297]:
def remove_fields(cols:list, columns_to_drop):

    cols2 = []

    for col in cols:
        if not col in columns_to_drop:
            cols2.append(col)
            # print(cols)

    return cols2

In [298]:
categoric_columns = remove_fields(categoric_columns, columns_to_drop) 
ordinal_columns = remove_fields(ordinal_columns, columns_to_drop)
cont_columns = remove_fields(cont_columns, columns_to_drop)

In [299]:
categoric_columns

['Ward_Type']

In [300]:
col_transformer = ColumnTransformer(
    transformers=[
        ('target', TargetEncoder(), categoric_columns),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ordinal_columns)
    ]
)

prknn_pipeline_list = [
    ('col_transform', col_transformer),
    ('scaler', MinMaxScaler()),
    ('knn_imputer', KNNImputer(n_neighbors=10, weights='distance')),
    ('prknn_predictor', PRKNeighborsClassifier(base_knn_params={"n_neighbors": 3}))
]

knn_pipeline_list = [
    ('col_transform', col_transformer),
    ('scaler', MinMaxScaler()),
    ('knn_imputer', KNNImputer(n_neighbors=10, weights='distance')),
    ('knn_predictor', KNeighborsClassifier(n_neighbors=3))
]


prknn_pipeline = Pipeline(prknn_pipeline_list)
knn_pipeline = Pipeline(knn_pipeline_list)

In [303]:

df_sample = df.sample(5_000)

X = df_sample.drop('Stay', axis=1)
y = df_sample['Stay']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=RANDOM_STATE)
ordinal_encoder = OrdinalEncoder()

y_train = ordinal_encoder.fit_transform(pd.DataFrame(y_train)).ravel()
y_test = ordinal_encoder.fit_transform(pd.DataFrame(y_test)).ravel()

# knn_pipeline.fit(X_train, y_train)
prknn_pipeline.fit(X_train, y_train)

# y_pred_knn = knn_pipeline.predict(X_test)
y_pred_prknn = prknn_pipeline.predict(X_test)

f1_score(y_test, y_pred_prknn, average='micro')


0.225